In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="bge-m3:567m",
)

In [3]:
# 載入 PDF 文件
loader = PyPDFLoader('./data/PDF_file.pdf')
docs = loader.load_and_split()

chunk_size = 256
chunk_overlap = 128

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
documents = text_splitter.split_documents(docs)

# 將文檔寫入 ChromaDB
db = Chroma.from_documents(
    documents,
    embedding=embeddings_model,
    persist_directory="./story-db" # bge-m3:567m
)

In [9]:
# Setup the Retriever
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

ollama_llm = OllamaLLM(
    base_url='http://dandelion-ollama-1:11434', 
    model="llama3.1:8b",
    temperature=0.0,
    num_predict=512
)

# 建立 RetrievalQA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    retriever=db.as_retriever(),
)

# 提問
query = "玫瑰是誰？"
result = qa_chain.invoke(query)
# print('提問:', query)
print("\n回答: ")
print(result)


回答: 
{'query': '玫瑰是誰？', 'result': '玫瑰是小王子的玫瑰花，還有園中其他五千朵玫瑰花（但小王子的玫瑰花是獨一無二的）。'}


In [10]:
qa_chain.invoke('最令你印象深刻的小王子的經歷是？')

{'query': '最令你印象深刻的小王子的經歷是？',
 'result': '最令我印象深刻的小王子的經歷是他無意中吐露的一些話逐漸使我搞清了他的來歷。'}

In [11]:
qa_chain.invoke('小王子的來歷是什？')

{'query': '小王子的來歷是什？', 'result': '小王子所來自的那個星球是小行星B612。'}

In [12]:
qa_chain.invoke('你是哪個模型？')

{'query': '你是哪個模型？', 'result': '我不知道。'}

In [15]:
qa_chain.invoke('誰喜歡喝酒？他出現在文章中哪裡？')

{'query': '誰喜歡喝酒？他出現在文章中哪裡？', 'result': '酒鬼。它出現在第十二段（XII）中。'}

In [16]:
qa_chain.invoke('你覺得小王子是個怎樣的人？')

{'query': '你覺得小王子是個怎樣的人？',
 'result': '根據文中描述，小王子的性格可以看出來。他似乎是一個敏感、浪漫、獨立的年輕人。他對他所遇到的陌生人的評價很細致，能夠看穿別人的真實面目。他也顯示出對自由和自主的渴望。'}

In [17]:
qa_chain.invoke('羊這個生物以怎樣的形式出現在文章中？為什出現？')

{'query': '羊這個生物以怎樣的形式出現在文章中？為什出現？',
 'result': '羊這個生物以畫的形式出現在文章中。它是小孩要求被描繪的東西，可能是因為小孩想學習畫畫或是對羊有特別的喜好。'}

In [18]:
# 顯示候選句
qa_chain = RetrievalQA.from_chain_type(
    llm=ollama_llm,
    retriever=db.as_retriever(),
    return_source_documents=True # show retrieve paragraphs
)

In [19]:
qa_chain.invoke('你覺得小王子是個怎樣的人？')

{'query': '你覺得小王子是個怎樣的人？',
 'result': '根據文中描述，小王子的性格可以看出來。他似乎是一個敏感、浪漫、獨立的年輕人。他對他所遇到的陌生人的評價很細致，能夠看穿別人的真實面目。他也顯示出對自由和自主的渴望。',
 'source_documents': [Document(id='212ebbb1-477a-4948-9c73-bff49e753ed1', metadata={'creationdate': 'D:20090501171152', 'moddate': 'D:20090501171152', 'creator': 'PrimoPDF http://www.primopdf.com', 'author': 'user', 'page_label': '27', 'page': 26, 'producer': 'AFPL Ghostscript 8.13', 'source': './data/PDF_file.pdf', 'total_pages': 54, 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)'}, page_content='小王子在他繼續往前旅行的途中，自言自語地說道： \n \n    “這個人一定會被其他那些人，國王呀，愛虛榮的呀，酒鬼呀，實業家呀， \n所瞧不起。 可是唯有他不使我感到荒唐可笑。 這可能是因為他所關心的是別的事， \n而不是他自己。” \n \n    他惋惜地嘆了口氣，并且又對自己說道： \n \n    “本來這是我唯一可以和他交成朋友的人。可是他的星球確實太小了，住不 \n下兩個人…” \n \n    小王子沒有勇氣承認的是：他留戀這顆令人贊美的星星，特別是因為在那里'),
  Document(id='bb8c854f-fd00-4ad0-a2be-07ec344f54b4', metadata={'moddate': 'D:20090501171152', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'creationdate': 'D:200

In [36]:
qa_chain.invoke('用你自己本身的創意，幫我寫出小王子的續集')

I don't know. The text only mentions the author's request to help him find out if the little prince has returned, but it doesn't provide any information about writing a sequel to the story.

{'query': '用你自己本身的創意，幫我寫出小王子的續集',
 'result': "I don't know. The text only mentions the author's request to help him find out if the little prince has returned, but it doesn't provide any information about writing a sequel to the story.",
 'source_documents': [Document(id='04220c5e-df03-4a0c-89ac-ea154ab67ee7', metadata={'total_pages': 54, 'author': 'user', 'creationdate': 'D:20090501171152', 'producer': 'AFPL Ghostscript 8.13', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'creator': 'PrimoPDF http://www.primopdf.com', 'moddate': 'D:20090501171152', 'page_label': '53', 'source': './data/PDF_file.pdf', 'page': 52}, page_content='果這時，有個小孩子向你走來，如果他笑著，他有金黃色的頭發，如果當你問他 \n問題時他不回答，你一定會猜得出他是誰。那就請你們幫個忙，不要讓我這么憂 \n傷：趕快寫信告訴我，他又回來了… \n \n \n                                                  「全文完」 \n \n \n                       關于《小王子》的作者 \n \n    聖﹒德克旭貝里（Antoine de Saiot-Exupery ） ，1900 年出生于法國里昂，'),
  Document(id='34605fc5-ba78-467b-91fe-9771083cbd89', metadata={'cr

In [33]:
# 啟動對話紀錄
chat_history = []

# 多輪問答測試
queries = [
    "這個故事告訴我們什麼道理？",
    "根據你的記憶, 不要埋怨誰？",
    "根據你的記憶, 為什麼？"
]

for query in queries:
    print(f"\n問題：{query}\n")
    print("\n回答：")
    result = qa_chain.invoke({"query": query, "chat_history": chat_history})
    
    print("\n ----------------- all result -----------------")
    # print(result)
    
    # 顯示檢索到的段落與相似度
    print("檢索到的段落 (含相似度)：")
    retriever = db.as_retriever()
    retrieved_docs_with_scores = db.similarity_search_with_score(query, k=4)
    for i, (doc, score) in enumerate(retrieved_docs_with_scores, 1):
        print(f"[{i}] 相似度: {score:.4f}")
        print(doc.page_content[:200] + "...")
        print("---")
    
    # 更新對話歷史
    chat_history.append((query, result))


問題：這個故事告訴我們什麼道理？


回答：
小孩子們對大人們應該寬厚些，不要埋怨他們。
 ----------------- all result -----------------
檢索到的段落 (含相似度)：
[1] 相似度: 0.7977
有一天他告訴我說：“我不該聽信她的話，絕不該聽信那些花兒的話，看看 
花，聞聞它就得了。我的那朵花使我的星球芳香四溢，可我不會享受它。關于老 
虎爪子的事，本應該使我產生同情，卻反而使我惱火…” 
 
    他還告訴我說： 
 
    “我那時什么也不懂！我應該根據她的行為，而不是根據她的話來判斷她。 
她使我的生活芬芳多彩，我真不該離開她跑出來。我本應該猜出在她那令人愛憐 
的花招后面所隱藏...
---
[2] 相似度: 0.8469
小孩子們對大人們應該寬厚些，不要埋怨他們。 
 
    當然，對我們懂得生活的人來說，我們才不在乎那些編號呢！我真愿意象講 
神話那樣來開始這個故事，我真想這樣說： 
 
    “從前呀，有一個小王子，他住在一個和他身體差不多大的星球上，他希望 
有一個朋友…”對懂得生活的人來說，這樣說就顯得真實。 
 
    我可不喜歡人們輕率地讀我的書。我在講述這些往事時心情是很難過的。我 
的朋友帶著...
---
[3] 相似度: 0.8744
羊。他想要一只小羊，這就証明他的存在。”他們一定會聳聳肩膀，把你當作孩 
子看待！但是，如果你對他們說：“小王子來自的星球就是小行星 B612 ”，那
么他們就十分信服，他們就不會提出一大堆問題來和你糾纏。他們就是這樣的。
小孩子們對大人們應該寬厚些，不要埋怨他們。 
 
    當然，對我們懂得生活的人來說，我們才不在乎那些編號呢！我真愿意象講 
神話那樣來開始這個故事，我真想這樣說： 
 
 ...
---
[4] 相似度: 0.8903
家的口吻來說話，可是猴面包樹的危險，大家都不大了解，對迷失在小行星上的 
人來說，危險性非常之大，因此這一回，我貿然打破了我的這種不喜歡教訓人的 
慣例。我說：“孩子們，要當心那些猴面包樹呀！”為了叫我的朋友們警惕這種 
危險──他們同我一樣長期以來和這種危險接觸， 卻沒有意識到它的危險性── 
我花了很大的功夫畫了這副畫。我提出的這個教訓意義是很重大的，花點功夫是 
很值得的。你們也許要問，為什...

In [32]:
print(chat_history)

[('這個故事告訴我們什麼道理？', {'query': '這個故事告訴我們什麼道理？', 'chat_history': [...], 'result': '小孩子們對大人們應該寬厚些，不要埋怨他們。', 'source_documents': [Document(id='67743f4f-00d3-481a-b625-81ef89d1b619', metadata={'producer': 'AFPL Ghostscript 8.13', 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\251.doc)', 'creator': 'PrimoPDF http://www.primopdf.com', 'page_label': '14', 'page': 13, 'author': 'user', 'total_pages': 54, 'creationdate': 'D:20090501171152', 'source': './data/PDF_file.pdf', 'moddate': 'D:20090501171152'}, page_content='有一天他告訴我說：“我不該聽信她的話，絕不該聽信那些花兒的話，看看 \n花，聞聞它就得了。我的那朵花使我的星球芳香四溢，可我不會享受它。關于老 \n虎爪子的事，本應該使我產生同情，卻反而使我惱火…” \n \n    他還告訴我說： \n \n    “我那時什么也不懂！我應該根據她的行為，而不是根據她的話來判斷她。 \n她使我的生活芬芳多彩，我真不該離開她跑出來。我本應該猜出在她那令人愛憐 \n的花招后面所隱藏的溫情。 花是多么自相矛盾！我當時太年青， 還不懂得愛她。 ”'), Document(id='209a46b2-5322-4195-b194-1a6a4e4095aa', metadata={'producer': 'AFPL Ghostscript 8.13', 'source': './data/PDF_file.pdf', 'total_pages': 54, 'title': '(Microsoft Word - LE PETIT PRINCE\\244\\244\\244\\345\\252\\